給你們看目前做了啥

Started by setting up the Python environment, checking versions, and installing the TorchCodec library.
Downloaded a sample dataset (dev-clean.tar.gz) from OpenSLR.org, which contains audio files from the LibriSpeech corpus.
The downloaded archive was extracted into a data/dev-clean directory.
Listed all .flac audio files within this extracted dataset.

Loading and Playing Individual Audio:

Loaded one of the .flac files using torchaudio to inspect its waveform shape and sample rate.
An IPython.display.Audio object was used to play this individual audio file directly in the notebook.

Creating an Audio Mixture:

To simulate a scenario where VAD might be useful (e.g., separating speakers), two different audio files were selected.
These two waveforms were then combined (averaged) to create a simple two-speaker audio mixture.
The resulting mixture was saved as data/mix_example.wav and also played in the notebook.

Voice Activity Detection (VAD) Function (energy_vad):

We defined a function called energy_vad that implements a simple energy-based VAD algorithm.

How it works:

Estimating Silence Level: The function first identifies frames with very low energy (considered 'silence-like') by looking at a lower percentile (e.g., 10%) of the energy distribution. It then calculates the average energy of these silence frames, establishing a 'noise floor'.
Thresholding for Speech: Speech is then detected in frames where the energy significantly exceeds this estimated noise floor (e.g., 4 times higher).
Handling Edge Cases: If almost the entire audio clip appears to be speech, it treats the whole clip as speech to avoid false negatives.
Converting to Time Segments: The function then converts these 'speech' frames into actual time segments (start and end times in seconds).

Merging Short/Close Segments: To create more robust and meaningful speech segments, it merges segments that are too short or are separated by very brief silences (e.g., less than 0.5 seconds).

Output: The energy_vad function returns a list of tuples, where each tuple (start_time, end_time) represents a detected speech segment in seconds.

Running VAD and Visualization:
We applied this energy_vad function to our mixed audio file (data/mix_example.wav).
The detected speech segments were then saved to a JSON file (outputs/vad_mix_example.json).
Finally, we visualized the original waveform with the detected speech segments highlighted in red, and played the segmented audio to hear the results.

In [ ]:
import os
import random
import torch
import torchaudio
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from tqdm.auto import tqdm
import IPython.display as ipd
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB_PLUS
# 假設你有引入 loss function，若無請確保你的環境有定義 pit_loss_si_sdr
# from your_utils import pit_loss_si_sdr

# ==========================================
# ★★★ 設定區域：在這裡控制人數 ★★★
# ==========================================
NUM_SPEAKERS = 3      # 想要做 5 人就改成 5
SAMPLE_RATE = 16000
DURATION = 3          # 每個切片 3 秒
DATA_ROOT = Path("data/LibriSpeech/dev-clean")     # 原始 flac 檔案的位置
OUT_ROOT = Path("data/dataset_multi")  # 生成資料的輸出位置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"目前設定為: {NUM_SPEAKERS} 人分離任務")
print(f"使用裝置: {DEVICE}")

目前設定為: 3 人分離任務
使用裝置: cuda


# Data from LibriSpeech ASR corpus
#### Sampling rate: 16kHz, 模型輸入必須使用 16 kHz
#### 如果模型需要噪音,可以搭配:MUSAN, 目前沒用上。

Downloading LibriSpeech dev-clean and unzip

In [ ]:
import os
import urllib.request
import tarfile  # <--- 記得 import 這個

os.makedirs("data", exist_ok=True)

url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
out_path = "data/dev-clean.tar.gz"

# 1. 下載
if not os.path.exists(out_path):
    print("Downloading sample data...")
    urllib.request.urlretrieve(url, out_path)
    print("Download Done.")
else:
    print("Already downloaded.")

# 2. 解壓縮 (補上這段)
# 檢查是否已經解壓縮過了 (檢查 data/LibriSpeech 是否存在)
if not os.path.exists("data/LibriSpeech"):
    print("Extracting data... (this may take a while)")
    with tarfile.open(out_path, "r:gz") as tar:
        tar.extractall("data")
    print("Extraction Done.")
else:
    print("Already extracted.")

Already downloaded.
Already extracted.


In [ ]:
# 安裝 torchcodec 和 soundfile (以防萬一)
!pip install torchcodec soundfile

In [ ]:
# 取得所有 .flac 檔案路徑
all_flac_files = list(Path(DATA_ROOT).rglob("*.flac"))
print(f"找到 {len(all_flac_files)} 個 FLAC 檔案")

def create_mix_dataset_dynamic(source_files, out_dir, n_speakers=2, n_samples=1000, sr=16000, duration=3):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    data_list = []
    target_len = int(sr * duration)

    print(f"正在生成 {n_samples} 筆資料，目標人數: {n_speakers} 人...")

    for i in tqdm(range(n_samples)):
        # 1. 隨機挑選 N 個檔案
        chosen_files = random.sample(source_files, n_speakers)

        mix_wav = torch.zeros(1, target_len)
        row = {} # 這一列 CSV 要存的資料

        # 2. 處理每一個 source 並加總
        for idx, file_path in enumerate(chosen_files):
            wav, _ = torchaudio.load(file_path)

            # 隨機裁切或補零到固定長度
            if wav.shape[1] > target_len:
                start = random.randint(0, wav.shape[1] - target_len)
                wav = wav[:, start:start+target_len]
            else:
                padding = target_len - wav.shape[1]
                wav = torch.nn.functional.pad(wav, (0, padding))

            # ★ 音量正規化：人越多，單一音量要越小，避免 Mix 爆音
            wav = wav / (wav.abs().max() + 1e-8) * (0.9 / n_speakers)

            # 儲存個別 Source (s1, s2, s3...)
            s_name = f"s{idx+1}_{i:05d}.wav"
            s_path = out_dir / s_name
            torchaudio.save(s_path, wav, sr)

            # 加到混合音訊中
            mix_wav += wav

            # 記錄路徑到 row
            row[f"s{idx+1}_path"] = str(s_path)

        # 3. 儲存 Mix
        mix_name = f"mix_{i:05d}.wav"
        mix_path = out_dir / mix_name
        torchaudio.save(mix_path, mix_wav, sr)

        row["mix_path"] = str(mix_path)
        data_list.append(row)

    # 存成 CSV
    df = pd.DataFrame(data_list)
    csv_path = out_dir.parent / f"meta_{n_speakers}spk_{out_dir.name}.csv"
    df.to_csv(csv_path, index=False)
    return csv_path

# --- 執行生成 (如果檔案已存在可以註解掉) ---
if len(all_flac_files) > 0:
    train_csv = create_mix_dataset_dynamic(all_flac_files, OUT_ROOT/"train", n_speakers=NUM_SPEAKERS, n_samples=10000)
    val_csv   = create_mix_dataset_dynamic(all_flac_files, OUT_ROOT/"val",   n_speakers=NUM_SPEAKERS, n_samples=200)
    print("資料生成完畢！")
else:
    print("錯誤：找不到來源音訊檔案，請確認 DATA_ROOT 設定")


找到 2703 個 FLAC 檔案
正在生成 10000 筆資料，目標人數: 3 人...


  0%|          | 0/10000 [00:00<?, ?it/s]

正在生成 200 筆資料，目標人數: 3 人...


  0%|          | 0/200 [00:00<?, ?it/s]

資料生成完畢！


#### 所有 .flac 音檔 save 在files

In [ ]:
class MixtureDataset(Dataset):
    def __init__(self, meta_csv, n_speakers=2):
        self.df = pd.read_csv(meta_csv)
        self.n_speakers = n_speakers

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # 讀取混合音訊
        mix, _ = torchaudio.load(row["mix_path"])

        # 動態讀取 N 個 Sources
        sources_list = []
        for i in range(1, self.n_speakers + 1):
            col_name = f"s{i}_path"
            s, _ = torchaudio.load(row[col_name])
            sources_list.append(s)

        # 堆疊: [N, Time]
        sources = torch.cat(sources_list, dim=0)

        return mix, sources

# 測試 DataLoader 是否正常
temp_dataset = MixtureDataset(train_csv, n_speakers=NUM_SPEAKERS)
temp_mix, temp_sources = temp_dataset[0]
print(f"Mix shape: {temp_mix.shape}, Sources shape: {temp_sources.shape}")
# 預期輸出 Sources shape 應該要是 [NUM_SPEAKERS, Time] (例如 [3, 48000])

Mix shape: torch.Size([1, 48000]), Sources shape: torch.Size([3, 48000])


In [ ]:
from torchaudio.models import ConvTasNet

class SeparationModel(nn.Module):
    def __init__(self, num_sources=2):
        super().__init__()
        # 針對 5-6 人分離，建議加大 kernel_size 或 layers
        # 這裡做了一些針對多人任務的微調建議
        self.model = ConvTasNet(
            num_sources=num_sources,
            enc_kernel_size=32,       # 加大 Kernel (原16 -> 32) 抓取更長特徵
            enc_num_feats=512,
            msk_num_feats=128,
            msk_num_hidden_feats=512,
            msk_num_layers=10,         # 如果做 5-6 人覺得效果不好，可改成 10
            msk_num_stacks=4
        )

    def forward(self, mix):
        return self.model(mix)

print(f"模型定義完成，支援 {NUM_SPEAKERS} 人分離")

模型定義完成，支援 3 人分離


In [ ]:
from itertools import permutations
import torch

def pit_loss_si_sdr(est, refs):
    """
    Permutation Invariant Training (PIT) Loss using SI-SDR
    Input:
        est:  [Batch, Speakers, Time]
        refs: [Batch, Speakers, Time]
    """
    B, C, T = est.shape

    # est: [B, C, 1, T]
    # refs: [B, 1, C, T]
    est_exp = est.unsqueeze(2)
    refs_exp = refs.unsqueeze(1)

    # --- SI-SDR 計算核心 ---
    eps = 1e-8

    # 1. 參考訊號的能量 (保持維度，方便後續廣播)
    # 形狀: [B, 1, C, 1]
    ref_energy = torch.sum(refs_exp ** 2, dim=-1, keepdim=True) + eps

    # 2. 投影 (Projection)
    # ★修正重點：這裡加了 keepdim=True，讓形狀變成 [B, C, C, 1]，才能跟 refs_exp [B, 1, C, T] 相乘
    dot = torch.sum(est_exp * refs_exp, dim=-1, keepdim=True)
    projection = dot * refs_exp / ref_energy

    # 3. 雜訊 (Noise)
    noise = est_exp - projection

    # 4. 信噪比 (SNR)
    # 同樣要 keepdim=True 或者是手動 unsqueeze 才能計算 ratio
    proj_energy = torch.sum(projection ** 2, dim=-1)
    noise_energy = torch.sum(noise ** 2, dim=-1) + eps

    ratio = proj_energy / noise_energy

    # 轉成 dB，形狀: [B, C_est, C_ref]
    pairwise_sdr = 10 * torch.log10(ratio + eps)

    # --- PIT: 尋找最佳排列組合 ---
    perms = list(permutations(range(C)))

    batch_losses = []
    for b in range(B):
        best_sdr = -float('inf')
        for p in perms:
            current_sdr = 0
            for i, j in enumerate(p):
                current_sdr += pairwise_sdr[b, i, j]

            if current_sdr > best_sdr:
                best_sdr = current_sdr

        batch_losses.append(-best_sdr / C)

    return torch.stack(batch_losses).mean()

print("Loss Function 修正完畢！請重新執行訓練。")

Loss Function 修正完畢！請重新執行訓練。


In [ ]:
# 1. 準備 DataLoaders
train_set = MixtureDataset(train_csv, n_speakers=NUM_SPEAKERS)
val_set   = MixtureDataset(val_csv,   n_speakers=NUM_SPEAKERS)

# 注意：多人分離時顯存消耗大，如果 OOM (Out of Memory)，請把 batch_size 調小成 2
BATCH_SIZE = 4
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_set,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 2. 初始化模型
model = SeparationModel(num_sources=NUM_SPEAKERS).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# 定義 Loss (這裡假設你有 pit_loss_si_sdr，若無可用 torchaudio 內建的)
# 如果之前的 code 有定義 pit_loss_si_sdr 就用之前的，沒有的話需要補上
from torch.nn.functional import mse_loss
def simple_pit_loss(est, target):
    # 這裡只是一個 placeholder，請使用你原本 notebook 裡的 pit_loss_si_sdr
    # 或是 torchaudio.functional.si_sdr 之類的
    return pit_loss_si_sdr(est, target)

EPOCHS = 30

# 建立模型儲存資料夾
import os
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

print(f"開始訓練 {EPOCHS} Epochs (每跑完一圈會自動存檔)...")
for epoch in range(1, EPOCHS + 1):
    model.train()
    train_loss = 0.0

    for mix, sources in tqdm(train_loader, desc=f"Epoch {epoch}"):
        mix = mix.to(DEVICE)
        sources = sources.to(DEVICE)

        optimizer.zero_grad()
        est = model(mix)

        # 對齊長度
        T_min = min(est.size(-1), sources.size(-1))
        est = est[..., :T_min]
        sources = sources[..., :T_min]

        # 計算 Loss (使用你原本定義的 loss function)
        loss = pit_loss_si_sdr(est, sources)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for mix, sources in val_loader:
            mix = mix.to(DEVICE)
            sources = sources.to(DEVICE)
            est = model(mix)

            T_min = min(est.size(-1), sources.size(-1))
            est = est[..., :T_min]
            sources = sources[..., :T_min]

            loss = pit_loss_si_sdr(est, sources)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    scheduler.step(avg_val_loss)

    print(f"[Epoch {epoch:02d}] Train Loss: {avg_train_loss:.3f} | Val Loss: {avg_val_loss:.3f}")

    # ★★★ 新增：存檔機制 ★★★
    # 儲存最好的模型 (Best Model)
    # 這裡簡單起見，每個 Epoch 都存一个，或者你可以加判断只存最好的
    torch.save(model.state_dict(), f"{save_dir}/model_epoch_{epoch}.pth")
    print(f"    💾 模型已儲存至 {save_dir}/model_epoch_{epoch}.pth")

開始訓練...


Epoch 1:   0%|          | 0/2500 [00:00<?, ?it/s]

[Epoch 01] Train Loss: 1.142 | Val Loss: 0.386


Epoch 2:   0%|          | 0/2500 [00:00<?, ?it/s]

[Epoch 02] Train Loss: -0.535 | Val Loss: -1.013


...

[Epoch 28] Train Loss: -8.503 | Val Loss: -6.012


Epoch 29:   0%|          | 0/2500 [00:00<?, ?it/s]

[Epoch 29] Train Loss: -8.587 | Val Loss: -6.166


Epoch 30:   0%|          | 0/2500 [00:00<?, ?it/s]

[Epoch 30] Train Loss: -8.717 | Val Loss: -6.380


In [ ]:
import numpy as np
model.eval()
import IPython.display as ipd

# 隨機挑一筆來聽
with torch.no_grad():
    dataset_check = MixtureDataset(val_csv, n_speakers=NUM_SPEAKERS)
    mix, sources = dataset_check[0]

    mix = mix.unsqueeze(0).to(DEVICE)      # [1, 1, T]
    est = model(mix)[0].cpu()              # [N, T]

    # ✅ 固定把需要的 numpy 變數留在 workspace，供後續 postprocess / ASR 直接用
    mix_np = mix[0, 0].cpu().numpy().astype(np.float32)      # [T]
    est_np = est.numpy().astype(np.float32)                  # [C, T]

    print(f"=== 混合音訊 (Mix) - {NUM_SPEAKERS} Speakers ===")
    ipd.display(ipd.Audio(mix_np, rate=SAMPLE_RATE))

    print("-" * 30)
    # 動態顯示每一位分離結果
    for i in range(NUM_SPEAKERS):
        est_i = est_np[i]
        print(f"🎧 Estimated Source {i+1}:")
        ipd.display(ipd.Audio(est_i, rate=SAMPLE_RATE))

=== 混合音訊 (Mix) - 3 Speakers ===


------------------------------
🎧 Estimated Source 1:


🎧 Estimated Source 2:


🎧 Estimated Source 3:


## ✅ Inference Post-processing：大幅減少「沙沙聲 / musical noise」+ 修正相位 + mixture consistency

你現在的情況（已能分離，但每路有雜音）很常見，**推論後處理**通常能立刻改善一大截，尤其是：

- **Polarity / 相位顛倒**：有些模型輸出可能整體乘上 -1（聽起來會更怪、也會影響之後的處理）
- **Mixture consistency**：理想上 `s1 + s2 + s3 = mix`。若不一致，容易出現殘留雜音/漏音
- **Global normalization**：不要每路各自 normalize（會把底噪感放大），要用同一個 gain

下面這段 code 你可以直接套在 `est = model(mix)` 之後，或用在你存成 wav 後再讀進來修正。


In [ ]:
import numpy as np

def _corr(a, b, eps=1e-12):
    a = a - np.mean(a)
    b = b - np.mean(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + eps))

def postprocess_separation(mix, est, weighted=True, fix_polarity=True, global_peak_norm=False):
    """
    mix: [T] float32
    est: [C, T] float32  (C speakers)
    returns: est_pp [C, T]
    """
    assert mix.ndim == 1 and est.ndim == 2
    C, T = est.shape
    assert T == mix.shape[0]

    est_pp = est.astype(np.float32).copy()

    # (1) Fix polarity if needed (sum(est) should correlate positively with mix)
    if fix_polarity:
        s = est_pp.sum(axis=0)
        if _corr(s, mix) < 0:
            est_pp *= -1.0

    # (2) Mixture consistency projection: force sum(est)=mix
    resid = mix - est_pp.sum(axis=0)  # [T]

    if weighted:
        w = np.abs(est_pp)
        w_sum = np.sum(w, axis=0, keepdims=True) + 1e-8
        est_pp = est_pp + (w / w_sum) * resid[None, :]
    else:
        est_pp = est_pp + resid[None, :] / C

    # (3) Global peak normalization (shared gain)
    if global_peak_norm:
        peak = float(max(np.max(np.abs(mix)), np.max(np.abs(est_pp))))
        if peak > 0:
            est_pp = est_pp / peak * 0.95

    return est_pp

# ---- optional: very lightweight STFT magnitude smoothing (reduces "musical noise") ----
def stft_smooth_denoise(x, sr, n_fft=1024, hop=256, smooth_time=5, smooth_freq=3):
    """
    A simple spectral smoothing trick:
    - STFT magnitude smoothing across time/freq
    - Keep original phase
    Notes:
    - This can reduce "沙沙點點" but may slightly blur consonants.
    """
    import numpy as np
    from scipy.ndimage import median_filter

    # frame
    pad = n_fft // 2
    xpad = np.pad(x, (pad, pad)).astype(np.float32)
    win = np.hanning(n_fft).astype(np.float32)

    frames = np.lib.stride_tricks.sliding_window_view(xpad, n_fft)[::hop].copy()
    frames = frames * win[None, :]

    X = np.fft.rfft(frames, axis=-1)
    mag = np.abs(X)
    phase = X / (mag + 1e-8)

    mag_s = median_filter(mag, size=(smooth_time, smooth_freq))
    X_s = mag_s * phase

    frames_s = np.fft.irfft(X_s, n=n_fft, axis=-1)

    # overlap-add
    out = np.zeros((frames_s.shape[0]*hop + n_fft,), dtype=np.float32)
    wsum = np.zeros_like(out)
    for i, fr in enumerate(frames_s):
        start = i * hop
        out[start:start+n_fft] += (fr.astype(np.float32) * win)
        wsum[start:start+n_fft] += win

    out = out / (wsum + 1e-8)
    out = out[pad: pad + len(x)]
    return out.astype(np.float32)


In [ ]:
import numpy as np
from scipy.signal import stft, istft
from scipy.ndimage import median_filter

def _fix_len(x, L):
    x = np.asarray(x)
    if x.shape[-1] > L:
        return x[..., :L]
    if x.shape[-1] < L:
        pad = [(0,0)] * x.ndim
        pad[-1] = (0, L - x.shape[-1])
        return np.pad(x, pad, mode="constant")
    return x

def tf_mask_reconstruct_with_mix_phase(mix, est, sr, n_fft=1024, hop=256, p=2.0,
                                       smooth_t=3, smooth_f=3):
    mix = mix.astype(np.float32)
    est = est.astype(np.float32)
    L = len(mix)

    # Make STFT/ISTFT invertible (avoid NOLA warning)
    f, tt, X = stft(
        mix, fs=sr, nperseg=n_fft, noverlap=n_fft-hop,
        window="hann", boundary="zeros", padded=True
    )

    mags = []
    for i in range(est.shape[0]):
        _, _, Si = stft(
            est[i], fs=sr, nperseg=n_fft, noverlap=n_fft-hop,
            window="hann", boundary="zeros", padded=True
        )
        mags.append(np.abs(Si) + 1e-8)
    M = np.stack(mags, axis=0)   # [C, F, T]

    Mp = M ** p
    mask = Mp / (Mp.sum(axis=0, keepdims=True) + 1e-8)

    # light smoothing to reduce musical noise
    mask = median_filter(mask, size=(1, smooth_f, smooth_t))

    Y = mask * X[None, :, :]     # [C, F, T] complex

    outs = []
    for i in range(est.shape[0]):
        _, yi = istft(
            Y[i], fs=sr, nperseg=n_fft, noverlap=n_fft-hop,
            window="hann", input_onesided=True, boundary=True
        )
        yi = _fix_len(yi.astype(np.float32), L)
        outs.append(yi)
    outs = np.stack(outs, axis=0)  # [C, L]

    # time-domain mixture consistency (exact sum)
    resid = mix - outs.sum(axis=0)
    w = np.abs(outs)
    outs = outs + (w / (w.sum(axis=0, keepdims=True) + 1e-8)) * resid[None, :]

    return outs


### ✅ 實際用法（把模型輸出 `est` 修一修再聽 / 再存 wav）

`ConvTasNet` 通常回傳 shape 類似 `[B, C, T]`（或 `[B, C, T]`），你可以在 `est = model(mix)` 後面這樣接：

- 先把 `mix`、`est` 轉 numpy
- `postprocess_separation(...)`
- （可選）對每一軌做 `stft_smooth_denoise(...)` 來減沙沙聲


In [ ]:
import numpy as np
from pathlib import Path

def _ensure_mix_est_np():
    """
    Ensure mix_np: [T], est_np: [C, T] exist in globals().
    Priority:
      1) existing mix_np & est_np
      2) existing torch/numpy variables: mix & est (common in many notebooks)
      3) load from wav files in current directory (mixed*.wav + estimated_source*.wav)
    """
    global mix_np, est_np, SAMPLE_RATE

    if "mix_np" in globals() and "est_np" in globals():
        return

    # Case 2: try variables named mix/est
    if "mix" in globals() and "est" in globals():
        try:
            import torch
            mix_t = mix
            est_t = est
            if isinstance(mix_t, torch.Tensor):
                mix_t = mix_t.detach().cpu()
            if isinstance(est_t, torch.Tensor):
                est_t = est_t.detach().cpu()

            mix_np = mix_t.squeeze().numpy().astype(np.float32)
            est_np = est_t.squeeze().numpy().astype(np.float32)

            if est_np.ndim == 1:
                est_np = est_np[None, :]
            # common shape: [T, C] -> transpose to [C, T]
            if est_np.shape[0] > est_np.shape[1]:
                # likely [T, C]
                est_np = est_np.T

            return
        except Exception as e:
            pass

    # Case 3: load from wav files
    try:
        import glob
        import soundfile as sf

        # prefer canonical names
        cand_mix = []
        for name in ["mixed(3 speakers).wav", "mixed.wav", "mix.wav"]:
            if Path(name).exists():
                cand_mix.append(name)
        if not cand_mix:
            cand_mix = sorted(glob.glob("mixed*.wav") + glob.glob("*mix*.wav"))

        cand_est = []
        for name in ["estimated_source_1.wav", "estimated_source_2.wav", "estimated_source_3.wav"]:
            if Path(name).exists():
                cand_est.append(name)
        if len(cand_est) < 2:
            cand_est = sorted(glob.glob("estimated_source_*.wav") + glob.glob("*source*.wav"))

        if not cand_mix or len(cand_est) < 2:
            raise FileNotFoundError("Need mix_np/est_np OR (mix & est) OR wav files (mixed*.wav + estimated_source_*.wav).")

        mix_wav, sr = sf.read(cand_mix[0])
        mix_wav = np.mean(mix_wav, axis=1).astype(np.float32) if mix_wav.ndim == 2 else mix_wav.astype(np.float32)

        est_list = []
        for p in cand_est:
            x, sr2 = sf.read(p)
            if sr2 != sr:
                raise ValueError(f"Sample rate mismatch: {p} has {sr2}, mix has {sr}")
            x = np.mean(x, axis=1).astype(np.float32) if x.ndim == 2 else x.astype(np.float32)
            est_list.append(x)

        # stack & align
        T = len(mix_wav)
        est_stack = np.stack([x[:T] if len(x) >= T else np.pad(x, (0, T-len(x))) for x in est_list], axis=0).astype(np.float32)

        mix_np = mix_wav
        est_np = est_stack
        if "SAMPLE_RATE" not in globals():
            SAMPLE_RATE = sr
        return

    except Exception as e:
        raise RuntimeError(
            "找不到可用的 mix_np / est_np。\n"
            "請先跑『分離推論那格』(會產生 mix_np / est_np)，或把 mixed*.wav / estimated_source*.wav 放在同目錄。\n"
            f"詳細錯誤：{type(e).__name__}: {e}"
        )

# ✅ 確保 mix_np / est_np 已定義
from pathlib import Path

# Run this once, then call _ensure_mix_est_np() anywhere you need mix_np/est_np.


In [ ]:
# (Optional) If you're on Google Colab, upload your wav files here:
# - mixed(3 speakers).wav (or mixed*.wav)
# - estimated_source_1.wav / estimated_source_2.wav / estimated_source_3.wav
try:
    from google.colab import files
    uploaded = files.upload()
    print("Uploaded:", list(uploaded.keys()))
except Exception as e:
    print("Not running in Colab (skip upload):", e)


Saving estimated_source_3.wav to estimated_source_3.wav
Saving estimated_source_2.wav to estimated_source_2.wav
Saving estimated_source_1.wav to estimated_source_1.wav
Saving mixed(3 speakers).wav to mixed(3 speakers).wav
Uploaded: ['estimated_source_3.wav', 'estimated_source_2.wav', 'estimated_source_1.wav', 'mixed(3 speakers).wav']


In [ ]:
# ✅ Load mixture + estimated sources
# This will use existing (mix_np, est_np) if already present,
# or load from wav files in the current directory (mixed*.wav + estimated_source_*.wav).
_ensure_mix_est_np()

# Quick sanity check
assert mix_np.ndim == 1, mix_np.shape
assert est_np.ndim == 2, est_np.shape
assert est_np.shape[1] == mix_np.shape[0], (est_np.shape, mix_np.shape)

print("Loaded mix_np:", mix_np.shape, "est_np:", est_np.shape, "SAMPLE_RATE:", SAMPLE_RATE)


Loaded mix_np: (48000,) est_np: (3, 48000) SAMPLE_RATE: 16000


In [ ]:
def gentle_gate(x, sr, th_db=-45.0, gamma=0.6):
    x = x.astype(np.float32)
    n = max(1, int(sr * 0.02))  # 20ms window
    pwr = np.convolve(x*x, np.ones(n, dtype=np.float32)/n, mode="same")
    rms = np.sqrt(pwr + 1e-12)
    th = 10 ** (th_db / 20.0)
    g = np.minimum(1.0, (rms / (th + 1e-12)) ** gamma)  # soft expander
    return (x * g).astype(np.float32)

In [ ]:
import numpy as np
import soundfile as sf

def save_multitrack_consistent(paths, tracks, sr, headroom_db=3.0):
    tracks = tracks.astype(np.float32)
    peak = np.max(np.abs(tracks))
    target = 10 ** (-headroom_db / 20.0)  # e.g., -3 dBFS
    g = target / (peak + 1e-12)
    tracks_out = tracks * g

    for i, p in enumerate(paths):
        sf.write(p, tracks_out[i], sr, subtype="PCM_24")


In [ ]:
def vad_mute(x, sr, th_db=-50, frame_ms=20, hop_ms=10, fade_ms=10):
    x = x.astype(np.float32)
    frame = int(sr*frame_ms/1000)
    hop = int(sr*hop_ms/1000)
    fade = int(sr*fade_ms/1000)

    n = 1 + max(0, (len(x)-frame)//hop)
    rms = np.zeros(n, np.float32)
    for i in range(n):
        seg = x[i*hop:i*hop+frame]
        rms[i] = np.sqrt(np.mean(seg*seg)+1e-12)

    thr = 10**(th_db/20)
    m = np.zeros(len(x), np.float32)
    for i in range(n):
        if rms[i] >= thr:
            m[i*hop:i*hop+frame] = 1.0

    # smooth edges
    if fade > 1:
        win = np.ones(fade, np.float32)
        m = np.convolve(m, win/win.sum(), mode="same")
        m = np.clip(m, 0, 1)

    return x*m

In [ ]:
# ✅ Final separation postprocess pipeline (artifact reduction + clean tails)

# Make sure mix_np / est_np exist:
_ensure_mix_est_np()
assert mix_np.ndim == 1 and est_np.ndim == 2
assert est_np.shape[1] == len(mix_np), (est_np.shape, mix_np.shape)

# 你們原本的後處理（polarity + mixture consistency + shared norm）
est_pp = postprocess_separation(mix_np, est_np, weighted=True)

# (1) 用 mix phase + soft mask 重建（大幅減少沙沙 / musical noise）
est_pp = tf_mask_reconstruct_with_mix_phase(
    mix_np, est_pp, SAMPLE_RATE,
    n_fft=1024, hop=256, p=2.0,
    smooth_t=3, smooth_f=3
)

# (2) 溫柔 gate（主要砍掉空白處尾巴底噪，不會硬切）
est_pp = np.stack([
    gentle_gate(est_pp[i], SAMPLE_RATE, th_db=-45, gamma=0.6)
    for i in range(est_pp.shape[0])
], axis=0).astype(np.float32)

# ✅ 只加強 speaker 3（index=2）
est_pp[2] = gentle_gate(est_pp[2], SAMPLE_RATE, th_db=-50)

# Final sanity
assert est_pp.shape[1] == len(mix_np), (est_pp.shape, mix_np.shape)
print("Final est_pp:", est_pp.shape)


Final est_pp: (3, 48000)


In [ ]:
import os
import numpy as np
import soundfile as sf
import IPython.display as ipd

# ✅ 如果你可能會跳著跑 cell，先保險補上 mix_np / est_np（能用就用）
if "mix_np" not in globals():
    try:
        _ensure_mix_est_np()
    except Exception as e:
        print("⚠️ _ensure_mix_est_np() failed:", e)

# ✅ 檢查 est_pp 是否存在
assert "est_pp" in globals(), "你還沒算出 est_pp：請先跑分離後處理那格（postprocess → mask → gate）"

# ✅ 用同一個 gain 存三軌，避免每軌 normalize 把沙沙聲放大
def save_multitrack_consistent(paths, tracks, sr, headroom_db=3.0):
    tracks = np.asarray(tracks, dtype=np.float32)
    peak = float(np.max(np.abs(tracks)))
    target = 10 ** (-headroom_db / 20.0)  # -3 dBFS
    g = target / (peak + 1e-12)
    tracks_out = tracks * g
    for i, p in enumerate(paths):
        sf.write(p, tracks_out[i], sr, subtype="PCM_16")
    return tracks_out

# 輸出檔名（你可以改）
out_paths = [f"post_spk{i+1}.wav" for i in range(est_pp.shape[0])]
est_pp_out = save_multitrack_consistent(out_paths, est_pp, SAMPLE_RATE, headroom_db=3.0)

print("✅ Saved postprocessed wavs:")
for p in out_paths:
    print(" -", os.path.abspath(p))

# ✅ 直接在 notebook 裡聽
for i in range(est_pp_out.shape[0]):
    print(f"🎧 Postprocessed Speaker {i+1}")
    ipd.display(ipd.Audio(est_pp_out[i], rate=SAMPLE_RATE))

#（可選）也把 mix 播放出來方便對照
if "mix_np" in globals():
    print("🎧 Mix (reference)")
    ipd.display(ipd.Audio(mix_np, rate=SAMPLE_RATE))


✅ Saved postprocessed wavs:
 - /content/post_spk1.wav
 - /content/post_spk2.wav
 - /content/post_spk3.wav
🎧 Postprocessed Speaker 1


🎧 Postprocessed Speaker 2


🎧 Postprocessed Speaker 3


🎧 Mix (reference)


In [21]:
import glob, os
paths = sorted(glob.glob("saved_models/model_epoch_*.pth"))
print("cwd =", os.getcwd())
print("found =", len(paths))
print("\n".join(paths[-10:]))  # 顯示最後 10 個

cwd = /content
found = 0



In [ ]:
print("peak before save:", np.max(np.abs(est_pp)))

save_multitrack_consistent(
    ["spk1.wav", "spk2.wav", "spk3.wav"],
    est_pp, SAMPLE_RATE, headroom_db=3.0
)

y,_ = sf.read("spk3.wav", dtype="float32")
print("peak after save:", np.max(np.abs(y)))

### ⚠️ (Deprecated) 不要在這裡重新 `sf.read(mixed...)` 覆蓋變數

你上面已經用 `_ensure_mix_est_np()` 統一處理載入了。  
如果你在這裡再讀一次 `mixed(3 speakers).wav / estimated_source_*.wav`，很容易讓後面 ASR/表格吃到「不是你最後 postprocess 的那版」。

✅ 正確做法：只在上面那格跑 `_ensure_mix_est_np()`，然後用 `est_pp` 做 ASR。


## ✅ 分離後接 ASR（語音轉文字）+ 順便做「說話者 classifier」

目標：
- 對每一軌分離語音做 ASR → 得到 transcript + timestamps
- 把每個片段再做 speaker embedding → 用 clustering / classifier 把片段標成 Speaker A/B/C
- 最後輸出一張表：`time | speaker | text`

> 注意：語音分離常見 **Permutation 問題**（source_1/2/3 可能互換）。  
> 用 embedding + clustering 可以自動對齊誰是誰，不用硬假設 source_1 永遠同一人。


In [ ]:
# 建議安裝（擇一）：
# 1) ASR: faster-whisper（快、CPU 也能跑）
!pip install -U faster-whisper

# 2) Speaker embedding: resemblyzer（簡單好用）
# !pip install -U resemblyzer

# 3) clustering
# !pip install -U scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

def run_asr_faster_whisper(wav_np, sr, language="zh", beam_size=5, model_size="small"):
    """
    Returns: list of segments: {start, end, text}
    """
    from faster_whisper import WhisperModel
    model = WhisperModel(model_size, device="cpu", compute_type="int8")

    segments, info = model.transcribe(
        wav_np,
        language=language,
        beam_size=beam_size,
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=300),
    )
    out = []
    for seg in segments:
        out.append({"start": float(seg.start), "end": float(seg.end), "text": seg.text.strip()})
    return out

def build_speaker_embeddings(segments_audio, sr):
    """
    segments_audio: list of 1D numpy arrays
    returns: [N, D] embeddings
    """
    from resemblyzer import VoiceEncoder, preprocess_wav
    enc = VoiceEncoder()
    embs = []
    for x in segments_audio:
        wav = preprocess_wav(x, source_sr=sr)
        embs.append(enc.embed_utterance(wav))
    return np.stack(embs, axis=0)

def kmeans_cluster(embs, k=3, seed=0):
    from sklearn.cluster import KMeans
    km = KMeans(n_clusters=k, random_state=seed, n_init="auto")
    labels = km.fit_predict(embs)
    return labels

def slice_audio(x, sr, start, end):
    a = int(max(0.0, start) * sr)
    b = int(min(float(len(x))/sr, end) * sr)
    return x[a:b]


### ✅ 用法：對每一軌跑 ASR → 收集所有片段 → embedding + KMeans 分 3 群（= 3 個說話者）

你要先準備：
- `est_pp`：分離後、postprocess 過的三軌（shape `[3, T]`）
- `SAMPLE_RATE`：例如 16000


In [ ]:
!pip -q install resemblyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 123.8 MB/s eta 0:00:00


In [ ]:
# Ensure mix_np exists (robust even if you run cells out of order)
import numpy as np

if "mix_np" not in globals():
    if "_ensure_mix_est_np" in globals():
        _ensure_mix_est_np()
    else:
        # Fallback: try loading from common wav filenames in current directory
        import glob
        from pathlib import Path
        import soundfile as sf

        cand = []
        for name in ["mixed(3 speakers).wav", "mixed.wav", "mix.wav"]:
            if Path(name).exists():
                cand.append(name)
        if not cand:
            cand = sorted(glob.glob("mixed*.wav") + glob.glob("*mix*.wav"))
        if not cand:
            raise NameError("mix_np is not defined. Run the separation/inference cell first, or put mixed*.wav in the working directory.")
        mix_np, sr0 = sf.read(cand[0], dtype="float32")
        if mix_np.ndim > 1:
            mix_np = mix_np.mean(axis=1)
        if "SAMPLE_RATE" not in globals():
            SAMPLE_RATE = int(sr0)

mix_segs = run_asr_faster_whisper(mix_np, SAMPLE_RATE, language=None)
pd.DataFrame(mix_segs)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

,start,end,text
0,0.0,3.0,I tell you the boy's blood will be on your head.


In [1]:
# Ensure est_pp exists (if you run cells out of order)
if "est_pp" not in globals():
    if "est_np" in globals() and "mix_np" in globals():
        # Minimal fallback: apply the final postprocess pipeline
        est_pp = postprocess_separation(mix_np, est_np, weighted=True)
        if "tf_mask_reconstruct_with_mix_phase" in globals():
            est_pp = tf_mask_reconstruct_with_mix_phase(mix_np, est_pp, SAMPLE_RATE,
                                                        n_fft=1024, hop=256, p=2.0,
                                                        smooth_t=3, smooth_f=3)
        if "gentle_gate" in globals():
            est_pp = np.stack([gentle_gate(est_pp[i], SAMPLE_RATE, th_db=-45, gamma=0.6)
                               for i in range(est_pp.shape[0])], axis=0)
    else:
        if "_ensure_mix_est_np" in globals():
            _ensure_mix_est_np()
            # est_np now exists
            est_pp = postprocess_separation(mix_np, est_np, weighted=True)
        else:
            raise NameError("est_pp is not defined. Run the final postprocess cell first.")

# est_pp: [C, T] 例如 C=3
C = est_pp.shape[0]

all_rows = []
all_seg_audio = []

# 1) 對每一軌做 ASR，取得 segments
for track in range(C):
    segs = run_asr_faster_whisper(est_pp[track], SAMPLE_RATE, language="zh", model_size="small")
    for s in segs:
        all_rows.append({
            "track": int(track),
            "start": float(s["start"]),
            "end": float(s["end"]),
            "text": s["text"],
        })
        all_seg_audio.append(slice_audio(est_pp[track], SAMPLE_RATE, s["start"], s["end"]))

df = pd.DataFrame(all_rows)
display(df.head())

# 2) 對每個 segment 抽 speaker embedding
embs = build_speaker_embeddings(all_seg_audio, SAMPLE_RATE)

# 3) KMeans 分群成 3 個 speaker（你可以把 k 改成你要的人數）
labels = kmeans_cluster(embs, k=3)
df["speaker_id"] = labels

# 4) 讓每個 track 自己「多數決」對應哪個 speaker（解 permutation）
major = df.groupby("track")["speaker_id"].agg(lambda x: int(x.value_counts().idxmax()))
track2spk = {int(t): int(s) for t, s in major.items()}
df["speaker"] = df["track"].map(lambda t: f"SPK{track2spk[int(t)]+1}")

# 5) 排時間，輸出成 diarization 風格表格
df = df.sort_values(["start", "end"]).reset_index(drop=True)
display(df.head(30))

# 6) 存成 CSV / JSON
df.to_csv("separation_asr_with_speaker.csv", index=False, encoding="utf-8-sig")
df.to_json("separation_asr_with_speaker.json", orient="records", force_ascii=False)

print("✅ Saved: separation_asr_with_speaker.csv / .json")


NameError: est_pp is not defined. Run the final postprocess cell first.

### ✅ 小提醒（讓 ASR + classifier 更穩、更不沙沙）
- **先做 VAD 再 ASR**：上面 faster-whisper 已經開 `vad_filter=True`
- **不要每路各自 normalize**（剛剛已經用 shared gain 解掉）
- **若仍有沙沙點點**：可以
  1) 在推論後加 `stft_smooth_denoise`（小幅，但立刻有效）
  2) 訓練時加 MR-STFT loss / complex mask（效果更好）
